In [ ]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from chempy import ReactionSystem
from chempy.kinetics.ode import get_odesys
%matplotlib inline

In [ ]:
system_str = """
  2 Fe+2 + H2O2 -> 2 Fe+3     + 2 OH-        ; 42
  2 Fe+3 + H2O2 -> 2 Fe+2     +   O2  + 2 H+ ; 17
       H+ + OH- ->   H2O                     ; 1e10
            H2O ->   H+       +   OH-        ; 1e-4
   Fe+3 + 2 H2O ->   FeOOH(s) + 3 H+         ; 1
FeOOH(s) + 3 H+ ->   Fe+3     + 2 H2O        ; 2.5
"""

In [ ]:
rsys = ReactionSystem.from_string(system_str)  # "[H2O]" = 1.0 (actually 55.4 at RT)
odesys, extra = get_odesys(rsys)
rsys

In [ ]:
tout = sorted(np.concatenate((np.linspace(0, 23), np.logspace(-8, 1))))
c0 = defaultdict(float, {'Fe+2': 0.05, 'H2O2': 0.1, 'H2O': 1.0, 'H+': 1e-7, 'OH-': 1e-7})
result = odesys.integrate(tout, c0, atol=1e-12, rtol=1e-14)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 6))
result.plot(names=[k for k in rsys.substances if k != 'H2O'], ax=axes[0])
axes[0].legend(loc='best', prop={'size': 10}); _ = axes[0].set_xlabel('Time'); _ = axes[0].set_ylabel('Concentration')
result.plot(names=[k for k in rsys.substances if k != 'H2O'], ax=axes[1], xscale='log', yscale='log')
axes[1].legend(loc='best', prop={'size': 10}); _ = axes[1].set_xlabel('Time'); _ = axes[1].set_ylabel('Concentration')
result.plot(deriv=True, names=[k for k in rsys.substances if k != 'H2O'], ax=axes[2])
axes[2].legend(loc='best', prop={'size': 10}); _ = axes[2].set_xlabel('Time'); _ = axes[2].set_ylabel('dC/dt')
result.plot_invariant_violations(ax=axes[3])
axes[3].legend(loc='best', prop={'size': 10}); _ = axes[3].set_xlabel('Time'); _ = axes[3].set_ylabel('Component')
plt.tight_layout()